In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import numpy as np
from sklearn import preprocessing,cross_decomposition,model_selection,metrics
import matplotlib.pyplot as plt
import seaborn as sns
# import xgboost as xgb
%matplotlib inline
import dateutil
from tqdm import tqdm
from sklearn import linear_model
import datetime
import numpy as np
import pandas as pd

import plotly.graph_objects as go
from ipywidgets import widgets
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels import tsa
import numpy as np
import pandas as pd

import plotly.graph_objects as go
from ipywidgets import widgets

In [4]:
confirmedGlobal=pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv',encoding='utf-8',na_values=None)

In [5]:
confirmedGlobal.head()


,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/28/21,12/29/21,12/30/21,12/31/21,1/1/22,1/2/22,1/3/22,1/4/22,1/5/22,1/6/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,157998,158037,158056,158084,158107,158189,158183,158205,158245,158275
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,208352,208899,208899,210224,210224,210885,210885,212021,212021,213257
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,217265,217647,218037,218432,218818,219159,219532,219953,220415,220825
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,22540,22823,23122,23740,23740,23740,24502,24802,25289,25289
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,76787,78475,79871,81593,82398,82920,83764,84666,86636,87625


In [6]:
bar_graph = confirmedGlobal[['Country/Region',confirmedGlobal.columns[-1]]].groupby('Country/Region').sum()

In [7]:
bar_graph = bar_graph.reset_index()
bar_graph.columns = ['Country/Region', 'values']

In [12]:
bar_graph = bar_graph.sort_values(by='values', ascending = False)
bar_graph['Country/Region'].values.tolist()

['US',
 'India',
 'Brazil',
 'United Kingdom',
 'France',
 'Russia',
 'Turkey',
 'Germany',
 'Italy',
 'Spain',
 'Iran',
 'Argentina',
 'Colombia',
 'Indonesia',
 'Poland',
 'Mexico',
 'Ukraine',
 'South Africa',
 'Netherlands',
 'Philippines',
 'Malaysia',
 'Czechia',
 'Canada',
 'Peru',
 'Thailand',
 'Belgium',
 'Iraq',
 'Vietnam',
 'Romania',
 'Chile',
 'Japan',
 'Bangladesh',
 'Portugal',
 'Switzerland',
 'Israel',
 'Greece',
 'Slovakia',
 'Sweden',
 'Serbia',
 'Austria',
 'Pakistan',
 'Hungary',
 'Kazakhstan',
 'Jordan',
 'Morocco',
 'Cuba',
 'Georgia',
 'Denmark',
 'Ireland',
 'Nepal',
 'United Arab Emirates',
 'Bulgaria',
 'Australia',
 'Lebanon',
 'Croatia',
 'Tunisia',
 'Belarus',
 'Korea, South',
 'Bolivia',
 'Guatemala',
 'Azerbaijan',
 'Sri Lanka',
 'Costa Rica',
 'Saudi Arabia',
 'Ecuador',
 'Lithuania',
 'Burma',
 'Panama',
 'Slovenia',
 'Paraguay',
 'West Bank and Gaza',
 'Venezuela',
 'Ethiopia',
 'Dominican Republic',
 'Uruguay',
 'Norway',
 'Kuwait',
 'Libya',
 'Mongo

In [11]:
bar_graph['values'].values.tolist()

[58487697,
 35226386,
 22328252,
 14100750,
 11290010,
 10420863,
 9789244,
 7447155,
 6975465,
 6922466,
 6203046,
 6025303,
 5242672,
 4264669,
 4179292,
 4055095,
 3869728,
 3504554,
 3309168,
 2888917,
 2776699,
 2510796,
 2436468,
 2329717,
 2245250,
 2206992,
 2095848,
 1843563,
 1827867,
 1817300,
 1741837,
 1589947,
 1499976,
 1455174,
 1448448,
 1422020,
 1389546,
 1375267,
 1327455,
 1312939,
 1301141,
 1276433,
 1074283,
 1047953,
 983629,
 970567,
 949176,
 929647,
 908672,
 830480,
 777584,
 770001,
 762458,
 753879,
 745544,
 735973,
 704907,
 657508,
 641817,
 631730,
 619502,
 590063,
 581394,
 568650,
 559950,
 532807,
 531624,
 512402,
 478860,
 472673,
 471414,
 445680,
 440024,
 439017,
 430958,
 428736,
 425455,
 392276,
 391657,
 390294,
 379402,
 378904,
 345389,
 306755,
 306686,
 297333,
 295601,
 287939,
 283214,
 283149,
 274087,
 260283,
 247863,
 246195,
 230265,
 229855,
 220825,
 220178,
 213257,
 203241,
 199507,
 192157,
 185237,
 183220,
 161965,
 1582

In [ ]:
country_names = bar_graph['Country/Region'].values.tolist()
cases = bar_graph['values'].values.tolist()

In [9]:
bar_graph

,Country/Region,values
182,US,58487697
79,India,35226386
23,Brazil,22328252
186,United Kingdom,14100750
62,France,11290010
...,...,...
112,Marshall Islands,7
149,Samoa,3
91,Kiribati,2
178,Tonga,1


In [10]:
# Total Cases Wordlwide
confirmedGlobal[confirmedGlobal.columns[-1]].sum()

300290277